## 분류평가
- TP, FP, FN, TN는 예측 클래스와 실제 클래스의 Positive 결정 값과 Negative 결정 값의 결합에 따라 결정
- 앞문자 TruelFalse는 예측값과 실제 값이 같은가 틀린가를 의미하고 뒤 문자 Negative/Positive는 예측 결과 값이 부정 긍정을 의미
- TN는 예측값을 Negative 값 0으로 예측했고 실제값 역시 Negative 값 이
- FP는 예측값을 Positive 값 1로 예측했고 실제 값은 Negative 값 이
- FN은 예측값을 Negative 값 0으로 예측했고 실제 값은 Positive 값 1
- TP는 예측값을 Positive 값 1로 예측했고 실제값 역시 Positive 값 1
- 정확도 = (TP + TN) / ( TP + TN + FP + FN)
- 정밀도 = TP / ( TP + FP)
- 재현율 = TPI (TP + FN)  *정밀도가 높으면 재현율이 낮아지는데 한쪽으로 치우쳐지지 않게 해야함
- F1 = 2 * ( 정밀도 * 재현을) / (정밀도 + 재현율) : 정밀도와 재현율이 어느 한쪽으로 치우치지 않는 수치를 나타낼때 높아짐.
- 정밀도와 재현율은 Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표
- 재현율이 중요 지표인 경우는 실제 Positive 양성 데이터를 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우(ex. 보험사기, 암진단)

## 정밀도 / 제현을 트레이드 오프
- 정밀도 또는 재현율이 특별히 강조돼야 할 경우 분류의 결정 임곗값(Threshold)을 조정해 정밀도 또는 재현율의 수치를 높일 수 있음
- 정밀도와 재현율은 한쪽을 높이면 다른 하나의 수치는 낮아지므로 트레이드오프(Trade-off)관계임

## 예측 확률을 반환하는 predict_proba()
- 사이킷런 분류 알고리즘은 예측 데이터가 특정 레이블(결정 클래스 값)에 속하는지를 계산하기 위해 먼저 개별 레이블별로 결정 확률을 구함
- 그리고 예측 확률이 큰 레이블 값으로 예측
- 이진 분류 모델에서 특정 데이터가 0이 될 확률이 10%, 1이 될 확률이 90%로 예측되었다면 최종 예측은 더 큰 확률을 가진 1로 예측
- 이진 분류에서는 이 임곗값을 0.5로 정하고 이 기준 값보다 확률이 크면 positive, 작으면 negative로 결정함
- 사이킷런은 객별 데이터별로 예측 확률을 반환하는 메서드인 predict_prob()를 제공 - 학습이 완료된 사이킷런 Classifier 객체에서 호출
- predict() 메서드와 유사하지만 단지 반환 결과가 예측 결과 클래스값이 아닌 예측 확률 결과임.

In [9]:
import pandas as pd
titanic_df = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/dataset/train.csv')
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [13]:
# 일괄 전처리 사용자 함수(null 처리, 불필요 칼럼 삭제, 레이블 인코딩) 
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
# Age(평균), Cabin('N'), Embarked('N'), Fare(0)
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
# PassengerId, Name, Ticket(티켓번호)
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행.
# Cabin(선실번호 첫문자만 추출 후 인코딩), Sex(성별), Embarked(중간 정착 항구)
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [17]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
from sklearn.metrics import accuracy_score

In [8]:
# 과제 
# 6개 알고리즘 적용, 학습/예측/평가를 수행
KNN, 서포트백터머신, 렌덤포레스트, decision tree, GBM, logistic Regression

# 6개 알고리즘에 대한 요약


In [18]:
from sklearn.model_selection import train_test_split
y_titanic_df = titanic_df['Survived']
X_titainc_df = titanic_df.drop('Survived', axis=1)
X_titainc_df = transform_features(X_titainc_df)

X_train, X_test, y_train, y_test = train_test_split(X_titainc_df, y_titanic_df,
test_size = 0.2, random_state=11)

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred=lr_clf.predict(X_test)
accuracy_lr = accuracy_score(y_test, pred)
accuracy_lr

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8491620111731844

In [23]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print(type(pred_proba[:3]))
print(pred.reshape(-1,1)[:3]) # 얘랑 마지막줄애랑 비교해서 보기 pred랑 pred_proba
print()
print(pred_proba[:3])

# 모든 행별로 1이 될지 0이될지를 나타내주는 거라고 생각하면 된다. 

<class 'numpy.ndarray'>
[[1]
 [0]
 [0]]

[[0.46200919 0.53799081]
 [0.87869718 0.12130282]
 [0.8771684  0.1228316 ]]
